# DOC
## LAST UPDATE: 2022-02-18
## IPDC PROCEDURE
## VISUALIZATION OF RESULTS

input: 

pickle files from 00_import
* "./data/pickle/H.gpickle"

pickle file from 02_DC, created after manual classification (see 02_DC notebook for instructions)
* "./data/pickle/gaps_final.pickle"


output:

folium map of classified gaps 
* "./results/gaps_foliumplot.html"

In [ ]:
# SETUP

%run -i packages.py

# import parameters for plotting
%run -i parameters_plot.py

# make subfolder for storing output
if not os.path.exists("./results"):
    os.mkdir("./results")

In [ ]:
# IMPORT OBJECTS FROM PREVIOUS STEPS

H = nx.read_gpickle("./data/pickle/H.gpickle")
gaps_final = pd.read_pickle("./data/pickle/gaps_final.pickle")

# NX attribute dicts (for plotting)
ced = nx.get_edge_attributes(H, "coord") # coordinates of edges dictionary ced
ted = nx.get_edge_attributes(H, "category_edge") # type of edges dictionary ted
tnd = nx.get_node_attributes(H, "category_node") # type of nodes dictionary tnd
cnd = nx.get_node_attributes(H, "coord") # coordinates of nodes dictionary cnd

In [ ]:
# add column for shortened address to be displayed on map
    
gaps_final["address_display"] = gaps_final["address"]

for i in gaps_final.index:
    mystr = gaps_final.at[i,"address"].split(", ")
    if len(mystr) > 1:
        gaps_final.at[i, "address_display"] = mystr[0] + ", ..."

In [ ]:
### PLOTTING OVERVIEW MAP

# Defining colors for gap classes: 
# bridges in red, missing links in orange, intersections in yellow,
# roundabouts in pink, rightturns in violet

gapcolors = {}
gapcolors["st"] = "#ffa208" ## for latex: \definecolor{myorange}{RGB}{255,162,8}
gapcolors["br"] = "#ff0000" ## for latex: \definecolor{myred}{RGB}{255, 0, 0}
gapcolors["rt"] = "#ff55f7" ## for latex: \definecolor{mypink}{RGB}{255,85,247}
gapcolors["ra"] = "#9400d3" ## for latex: \definecolor{myviolet}{RGB}{148,0,211}
gapcolors["is"] = "#fae700" ## for latex: \definecolor{myyellow}{RGB}{250,231,0}

mycity_coord = [55.6761, 12.5683]

m = folium.Map(location = mycity_coord, zoom_start = 13, tiles = None) 


# add tile layers to m
for key in ["Google Satellite", "osm", "st_lite_op3", "white_background"]:
    basemaps[key].add_to(m)


leg_classes = {"br": "Bridge",
               "is": "Intersection",
               "st": "Street",
               "rt": "Right turn",
               "ra": "Roundabout"}

### Info pop up feature group:

my_fg_info = folium.FeatureGroup("Gap info", show = False)
my_fg_ranktext = folium.FeatureGroup("Gap rank", show = False)
my_fg_address = folium.FeatureGroup("Address", show = False)

for i in range(len(gaps_final)):
    
    mygap = gaps_final.loc[i]
    
    mygapnumber = mygap["gapnumber"]
    myclass = mygap["class"]
    myclass_full = leg_classes[myclass]
    
    mycolor = gapcolors[myclass]
    
    mycoords = mygap["coord"]
    myaddress = mygap["address"]
    myrank = mygap["rank"]
    
    myaddress_display = mygap["address_display"]
    
    ### INFO POP UP
    myinfo = "Gap " + str(mygapnumber) + " (" + myclass_full +  "), " + myaddress + ". Benefit: " + str(int(np.round(myrank)))
    my_marker = folium.Marker(location = mycoords[0][0], 
                              popup = myinfo).add_to(my_fg_info)
    
    
    ### GAP RANK AS TEXT
    folium.Marker(mycoords[0][0],
    icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 15pt; font-weight: bold, font-family:georgia"><p style="color: #000000; font-weight: bold">' + str(mygapnumber) + '</p></div>')).add_to(my_fg_ranktext)

    ### GAP ADDRESS AS TEXT
    folium.Marker(mycoords[0][0],
    icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(-25,0),
        html='<div style="font-size: 15pt; font-weight: bold, font-family:georgia"><p style="color: #000000; font-weight: bold">' + str(myaddress_display) + '</p></div>')).add_to(my_fg_address)


my_fg_info.add_to(m)
my_fg_ranktext.add_to(m)
my_fg_address.add_to(m)

##### streets and bikeable paths (protected infrastructure) #####
snw = folium.FeatureGroup(name = "All streets", show = True)
bnw2 = folium.FeatureGroup(name = "Bike paths", show = True) # bikeable edges (bike and multi)

sloc = []
bloc = []

for edge in H.edges:
    myloc = [(c[1], c[0]) for c in ced[tuple(sorted(edge))].coords]
    sloc.append(myloc)
    if ted[edge] != "car":
        bloc.append(myloc)
        
snw_line = folium.PolyLine(locations = sloc, weight = 1, color = "#dadada").add_to(snw)
bnw_line = folium.PolyLine(locations = bloc, weight = 2, color = "black").add_to(bnw2)

snw.add_to(m)
bnw2.add_to(m)

### One feature group for each class

for myclass in np.unique(gaps_final["class"]):
    myclass_fullname = "Gaps: " + leg_classes[myclass]
    mycolor = gapcolors[myclass]
    mycoords = [item for sublist in gaps_final[gaps_final["class"]==myclass]["coord"] for item in sublist]
    myfg = folium.FeatureGroup(myclass_fullname, show = True)
    mycontour = folium.PolyLine(locations = mycoords, weight = 8, color = "black").add_to(myfg)
    myline = folium.PolyLine(locations = mycoords, weight = 6, color = mycolor).add_to(myfg)
    myfg.add_to(m)

# to add a legend as png image layer to folium plot:
# from folium import plugins
# plugins.FloatImage(image = "legend2.png", bottom=5, left=5).add_to(m)

### Layer control    
folium.LayerControl().add_to(m)

#m.save(mname)
m.save("./results/gaps_classified_plot.html")

## END OF 03_PLOT NOTEBOOK